In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from technical_analysis.utils.utils import *

c:\Users\b90361388\Desktop\sst_iteso\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("Notebook/files/Binance_BTCUSDT_1h.csv", header=1)
data.rename(columns={'Volume BTC':'Volume'}, inplace=True)

data.index = pd.to_datetime(data['Date'], format="ISO8601")
data.sort_index(inplace=True)
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume,Volume USDT,tradecount
Date,,,,,,,,,,
2017-08-17 04:00:00,1502942400000,2017-08-17 04:00:00,BTCUSDT,4261.48,4313.62,4261.32,4308.83,47.181009,2.023661e+05,171
2017-08-17 05:00:00,1502946000000,2017-08-17 05:00:00,BTCUSDT,4308.83,4328.69,4291.37,4315.32,23.234916,1.003048e+05,102
2017-08-17 06:00:00,1502949600000,2017-08-17 06:00:00,BTCUSDT,4330.29,4345.45,4309.37,4324.35,7.229691,3.128231e+04,36
2017-08-17 07:00:00,1502953200000,2017-08-17 07:00:00,BTCUSDT,4316.62,4349.99,4287.41,4349.99,4.443249,1.924106e+04,25
2017-08-17 08:00:00,1502956800000,2017-08-17 08:00:00,BTCUSDT,4333.32,4377.85,4333.32,4360.69,0.972807,4.239504e+03,28
...,...,...,...,...,...,...,...,...,...,...
2025-09-22 19:00:00,1758567600000,2025-09-22 19:00:00,BTCUSDT,112429.12,112600.87,111936.40,112122.90,1307.373650,1.467768e+08,126232
2025-09-22 20:00:00,1758571200000,2025-09-22 20:00:00,BTCUSDT,112122.90,112977.41,111975.28,112781.88,596.840050,6.707508e+07,93553
2025-09-22 21:00:00,1758574800000,2025-09-22 21:00:00,BTCUSDT,112781.87,112970.00,112602.79,112969.99,293.311560,3.307493e+07,42931


In [19]:
# # --- 1. Definir el grid de parámetros para la optimización ---
# Añadimos una nueva entrada para la estrategia 'ma' con su parámetro 'window'.
param_grid = {
    "ma": {
        'window': [6, 12] # Parámetros para la Media Móvil
    },
    'n_shares': [1, 3]
}

# --- 2. Definir qué combinaciones de estrategias quieres probar ---
# Ahora Optuna probará tres configuraciones:
# - RSI por sí solo.
# - MA por sí solo.
# - Una estrategia compuesta donde RSI y MA deben estar de acuerdo.
strategies_to_combine = [['ma']]

# --- 3. Dividir los datos y ejecutar la optimización (Sin cambios) ---
train_size = int(len(data) * 0.7)
train_df = data.iloc[:train_size]
validation_df = data.iloc[train_size:]

best_params = optimize_hyperparameters(
    param_grid,
    strategies_to_combine,
    train_df,
    validation_df,
    n_trials=50 # Puedes aumentar los trials para una búsqueda más completa
)

# --- 4. Construir la mejor estrategia (Sin cambios) ---
# Este bloque funciona sin modificaciones, ya que interpreta dinámicamente
# los resultados de 'best_params'.
best_strat_names = eval(best_params['strategy_combination'])
best_strategy_instances = []
for name in best_strat_names:
    params = {k.split('_', 1)[1]: v for k, v in best_params.items() if k.startswith(name)}
    best_strategy_instances.append(STRATEGY_MAPPING[name](**params))

if len(best_strategy_instances) > 1:
    final_strategy = CompoundStrategy(strategies=best_strategy_instances)
else:
    final_strategy = best_strategy_instances[0]

final_n_shares = best_params['n_shares']
final_sl_tp = (
    best_params['sl_long_factor'],
    best_params['tp_long_factor'],
    1 + (1 - best_params['sl_long_factor']),
    1 - (best_params['tp_long_factor'] - 1)
)


# --- 5. Ejecutar una validación cruzada final (Sin cambios) ---
cv_backtester = CrossValidationBacktester(full_data=data, n_splits=5)
results = cv_backtester.run_cv(
    strategy=final_strategy,
    n_shares=final_n_shares,
    sl_tp_factors=final_sl_tp
)

# --- 6. Visualizar los resultados (Sin cambios) ---
plot_backtesting_results(results)

[I 2025-10-07 15:46:26,456] A new study created in memory with name: no-name-613134ad-4657-4acc-8400-ce35f2c46ff5
[I 2025-10-07 15:46:27,762] Trial 0 finished with value: 279.61026173027386 and parameters: {'strategy_combination': "['ma']", 'ma_window': 12, 'n_shares': 3, 'sl_long_factor': 0.9797730459999549, 'tp_long_factor': 1.0433608030790704}. Best is trial 0 with value: 279.61026173027386.
[I 2025-10-07 15:46:29,069] Trial 1 finished with value: 196.41991701199822 and parameters: {'strategy_combination': "['ma']", 'ma_window': 9, 'n_shares': 3, 'sl_long_factor': 0.9546493593377555, 'tp_long_factor': 1.0715074956230481}. Best is trial 0 with value: 279.61026173027386.
[I 2025-10-07 15:46:30,245] Trial 2 finished with value: 232.9776193630893 and parameters: {'strategy_combination': "['ma']", 'ma_window': 11, 'n_shares': 2, 'sl_long_factor': 0.9569282284965339, 'tp_long_factor': 1.084139869017656}. Best is trial 0 with value: 279.61026173027386.
[I 2025-10-07 15:46:31,516] Trial 3 f

Mejores parámetros encontrados: {'strategy_combination': "['ma']", 'ma_window': 9, 'n_shares': 1, 'sl_long_factor': 0.9866138049186541, 'tp_long_factor': 1.0393642024129173}
Ejecutando Fold 1/5...
Ejecutando Fold 2/5...
Ejecutando Fold 3/5...
Ejecutando Fold 4/5...
Ejecutando Fold 5/5...


In [20]:
from performance_metrics.metrics import win_rate, maximum_drawdown

last_fold_idx = max(results.keys())
last_fold_df = results[last_fold_idx]

metrics = {}
for freq_label, freq_str in zip(['monthly', 'quarterly', 'yearly'], ['M', 'Q', 'Y']):
    resampled = last_fold_df['portfolio_value'].resample(freq_str)
    period_values = [group for _, group in resampled]
    win_rates = [win_rate(pv) for pv in period_values]
    drawdowns = [maximum_drawdown(pv) for pv in period_values]
    metrics[freq_label] = {
        'win_rate': win_rates,
        'maximum_drawdown': drawdowns
    }

metrics

{'monthly': {'win_rate': [0.24383561643835616,
   0.20305980528511822,
   0.3149394347240915,
   0.3162853297442799,
   0.23226703755215578,
   0.253028263795424,
   0.3379694019471488,
   0.27860026917900405,
   0.3001345895020188,
   0.19913419913419914,
   0.21987951807228914,
   0.31529850746268656,
   0.2654494382022472,
   0.2351233671988389,
   0.2787456445993031,
   0.247557003257329,
   0.27722772277227725],
  'maximum_drawdown': [-0.0013993545201219648,
   -2.220446049250313e-16,
   -0.0022721453665369706,
   -0.00019958491463401007,
   -0.00014627958087454207,
   -0.00022736527425570863,
   8.512439183128162e-05,
   0.0,
   0.0,
   -5.3834726314594405e-06,
   0.0,
   -1.1102230246251565e-16,
   -2.220446049250313e-16,
   -5.785610774444727e-05,
   0.0,
   -0.00018590234354609603,
   -0.0003835196216590564]},
 'quarterly': {'win_rate': [0.21658986175115208,
   0.2881739918441323,
   0.28953330312641595,
   0.2348968105065666,
   0.26817947395564723,
   0.26666666666666666],
 

In [21]:
monthly = pd.DataFrame(metrics['monthly'])
quarterly = pd.DataFrame(metrics['quarterly'])
yearly = pd.DataFrame(metrics['yearly'])

In [22]:
monthly

,win_rate,maximum_drawdown
0,0.243836,-1.399355e-03
1,0.203060,-2.220446e-16
2,0.314939,-2.272145e-03
3,0.316285,-1.995849e-04
4,0.232267,-1.462796e-04
5,0.253028,-2.273653e-04
6,0.337969,8.512439e-05
7,0.278600,0.000000e+00
8,0.300135,0.000000e+00
9,0.199134,-5.383473e-06


In [23]:
quarterly

,win_rate,maximum_drawdown
0,0.216590,-1.399355e-03
1,0.288174,-2.272145e-03
2,0.289533,-2.273653e-04
3,0.234897,0.000000e+00
4,0.268179,-1.110223e-16
5,0.266667,0.000000e+00


In [24]:
yearly

,win_rate,maximum_drawdown
0,0.274677,-0.001399
1,0.253611,0.000000
